In [1]:
import pandas as pd
from selenium import webdriver

In [2]:
ord_otp = pd.read_csv('../data/ORD_OTP.csv')

In [3]:
unique_arr_flights = ord_otp[['OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM']].drop_duplicates()

In [4]:
unique_arr_flights.shape

(10164, 2)

In [29]:
lambda x: ord_otp['FL_DATE'][0][:4]

'2013'

In [5]:
ord_otp['YEAR'] = list(map(lambda x: x[:4], ord_otp['FL_DATE']))

In [6]:
ord_otp.query('YEAR == "2017"')['TAIL_NUM'].unique().shape

(3132,)

In [7]:
## unique_arr_flights['OP_UNIQUE_CARRIER'].unique()

In [8]:
airlines = pd.read_csv('../data/airlines.csv')

In [9]:
airlines

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [10]:
extra = pd.DataFrame({'IATA_CODE': ['9E','YV','OH', 'YX'], 'AIRLINE': ['Endeavor Air', 'Mesa Airlines', 'PSA Airlines', 'Republic Airways']})

In [11]:
airlines = airlines.append(extra,ignore_index=True)

In [12]:
airlines

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [13]:
unique_arr_flights.head()

,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM
0,AA,618
4,AA,153
8,AA,506
12,AA,510
16,AA,512


In [14]:
merge1 = unique_arr_flights.set_index(['OP_UNIQUE_CARRIER']).join(airlines.set_index(['IATA_CODE'])).reset_index()

In [15]:
merge1.head()

,index,OP_CARRIER_FL_NUM,AIRLINE
0,9E,3309,Endeavor Air
1,9E,3312,Endeavor Air
2,9E,3698,Endeavor Air
3,9E,3561,Endeavor Air
4,9E,3728,Endeavor Air


In [16]:
driver = webdriver.Chrome()
url_ = 'https://www.seatguru.com/'
driver.get(url_)

In [17]:
#POPUP SCREEN EXIT BUTTON PATH
path1 = '/html/body/div[7]/div/div/div/div/div[1]/img'
exit_button = driver.find_element_by_xpath(path1)
exit_button.click()

In [18]:
airline_input = driver.find_element_by_id('airline-select')
airline_input.get_attribute('class')

'autocomplete native-input ui-autocomplete-input'

In [22]:
airline_input.clear()
# airline_input.send_keys('American Airlines')
# airline_input.text

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=80.0.3987.132)


In [18]:
flight_num_input = driver.find_element_by_id('flightno')
# flight_num_input.clear()
flight_num_input.send_keys('618')

In [20]:
##LOCATE SEARCH BUTTON
search_button = driver.find_element_by_id('search')
search_button.click()


In [44]:
merge1.shape

(10164, 3)

---

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time

In [73]:
airlines_url = "https://www.seatguru.com/browseairlines/browseairlines.php"
airlines_soup = BeautifulSoup(requests.get(airlines_url).text, 'html.parser')

**Code below takes about 3 minutes to scrape:**

In [116]:
base_url = "https://www.seatguru.com"
airline_name = []
IATA_code = []
plane_name = []
seat_count = []
# Iterate through airlines
for a in airlines_soup.find("div", {"class":"browseAirlines"}).select('li a'):
    airline_url = a['href']
    airline_soup = BeautifulSoup(requests.get(base_url + airline_url).text, 'html.parser')
    airline_code = airline_soup.find("div", {"class":"sidebar-aircraft"}).select('span.ai-label:contains("Airline Code:") + span')[0].text
    # Iterate through tables on airline page
    for tbl in airline_soup.find_all("table", {"class": "seats"}):
        # Iterate through table rows
        for tr in tbl.find_all('tr'):
            # Reset seat total
            seats = 0
            # Iterate through seat types adding them up
            for s in tr.find_all('span', {"class": "seat_count"}):
                seats += int(s.text)
            plane_name.append(tr.text)
            seat_count.append(seats)
            airline_name.append(a.text)
            IATA_code.append(airline_code)
    time.sleep(0.1)

In [118]:
df = pd.DataFrame({'airline' : airline_name, 'IATA_code' : IATA_code, 'plane' : plane_name, 'seat_count' : seat_count})
df['plane'] = df['plane'].explode()
df['plane'].replace(regex=True,inplace=True,to_replace=r'\n',value=r'')
df['plane'] = df['plane'].str.split("Seats").str[0]

In [119]:
df

,airline,IATA_code,plane,seat_count
0,Aegean Airlines,A3,Airbus A319 (319),138
1,Aegean Airlines,A3,Airbus A320 (320),174
2,Aegean Airlines,A3,Airbus A321 (321),196
3,Aegean Airlines,A3,ATR 42-600 (ATR),48
4,Aegean Airlines,A3,Bombardier Q400 (DH4),78
...,...,...,...,...
1275,Xiamen Airlines,MF,Boeing 737-800 (738) Layout 2,170
1276,Xiamen Airlines,MF,Boeing 737-800 (738) Layout 3,164
1277,Xiamen Airlines,MF,Boeing 737-800 (738) One Class,184
1278,Xiamen Airlines,MF,Boeing 757-200 (752) Layout 1,174


In [122]:
df.to_csv("../data/seat_counts_seat_guru.csv")